In [1]:
!pip install cakelens-v5

In [2]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.3 MB/s eta 0:00:00


In [3]:
import pathlib
from cakelens.detect import Detector
from cakelens.model import Model
import torch

# Create model and load from Hugging Face Hub
model = Model()
# load the model weights from Hugging Face Hub
model.load_from_huggingface_hub(device=torch.device('cpu'))

cakelens-model-5.0.pth:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

In [4]:
# Create detector
detector = Detector(
    model=model,
    batch_size=1,
    device="cuda"  # or "cuda", "mps", or None for auto-detection
)

# Run detection
video_path = pathlib.Path("/content/Deepfake_Video_Generation_For_Detector_Testing.mp4")
verdict = detector.detect(video_path)

# Access results
print(f"Video: {verdict.video_filepath}")
print(f"Frame count: {verdict.frame_count}")
print("Predictions:")
for i, prob in enumerate(verdict.predictions):
    print(f"  Label {i}: {prob * 100:.2f}%")

Video: /content/Deepfake_Video_Generation_For_Detector_Testing.mp4
Frame count: 192
Predictions:
  Label 0: 96.40%
  Label 1: 41.63%
  Label 2: 8.09%
  Label 3: 1.46%
  Label 4: 0.35%
  Label 5: 67.32%
  Label 6: 0.09%
  Label 7: 2.27%
  Label 8: 0.00%
  Label 9: 16.42%
  Label 10: 77.38%
  Label 11: 1.15%
  Label 12: 0.18%
  Label 13: 1.92%
  Label 14: 0.69%
  Label 15: 8.96%
  Label 16: 6.70%


In [14]:
pip install fastapi uvicorn torch

In [12]:
import pathlib
import torch
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from cakelens.detect import Detector
from cakelens.model import Model

# Initialize FastAPI app
app = FastAPI()

In [13]:
detector = Detector(
    model=model,
    batch_size=1,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

@app.post("/detect")
async def detect_video(file: UploadFile = File(...)):
    # Save uploaded file locally
    video_path = pathlib.Path(f"./{file.filename}")
    with open(video_path, "wb") as buffer:
        buffer.write(await file.read())

    # Run detection
    verdict = detector.detect(video_path)

    # Clean up (optional: delete uploaded file after detection)
    video_path.unlink(missing_ok=True)

    # Return only predictions
    return JSONResponse(content={
        "predictions": verdict.predictions
    })

In [17]:
%%writefile app.py
import pathlib
import torch
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from cakelens.detect import Detector
from cakelens.model import Model

# Initialize FastAPI app
app = FastAPI()

# Create model and load from Hugging Face Hub
model = Model()
# load the model weights from Hugging Face Hub
model.load_from_huggingface_hub(device=torch.device('cpu'))

detector = Detector(
    model=model,
    batch_size=1,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

@app.post("/detect")
async def detect_video(file: UploadFile = File(...)):
    # Save uploaded file locally
    video_path = pathlib.Path(f"./{file.filename}")
    with open(video_path, "wb") as buffer:
        buffer.write(await file.read())

    # Run detection
    verdict = detector.detect(video_path)

    # Clean up (optional: delete uploaded file after detection)
    video_path.unlink(missing_ok=True)

    # Return only predictions
    return JSONResponse(content={
        "predictions": verdict.predictions
    })

Writing app.py


In [20]:
!pip install pyngrok

In [21]:
!ngrok http 8000

ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  https://ngrok.com/docs/errors/err_ngrok_4018
ERROR:  


In [26]:
from pyngrok import ngrok, conf

# Set your authtoken (replace with your token)
conf.get_default().auth_token = "322RC3srUCGegxIyNGmDOsXwwtv_3yDSkv9uynQbhkWaGei1V"

In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

import pathlib
import torch
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from cakelens.detect import Detector
from cakelens.model import Model
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# --- Fix Colab event loop issue ---
nest_asyncio.apply()

# --- Initialize FastAPI app ---
app = FastAPI()

# Load model once
model = Model()
model.load_from_huggingface_hub(device=torch.device("cpu"))

detector = Detector(
    model=model,
    batch_size=1,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

@app.post("/detect")
async def detect_video(file: UploadFile = File(...)):
    video_path = pathlib.Path(f"./{file.filename}")
    with open(video_path, "wb") as buffer:
        buffer.write(await file.read())

    verdict = detector.detect(video_path)
    video_path.unlink(missing_ok=True)

    return JSONResponse(content={
        "predictions": verdict.predictions
    })

# --- Expose the API via ngrok ---
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# --- Run Uvicorn (non-blocking in Colab) ---
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [2807]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://b3028f71bc80.ngrok-free.app" -> "http://localhost:8000"
